# Data preparation

This notebook is aimed to prepare the data for fitting and assessing DeeProtGO when predicting protein function annotations represented by Biological Process (BP) terms of Gene Ontology (GO). Data for model training and the benchmark data using CAFA3 *NK* proteins from eukaryotic organisms is here prepared. 

**Note**: Running this notebook could take some hours. If you only want to train and test DeeProtGO, without running this notebook, you can go to the training and testing scripts and use the data is provided in the repository.

## Requirements

In [ ]:
!pip install -U pandas-profiling > /dev/null 2>&1
!pip install -U "bio-embeddings[all]" > /dev/null 2>&1
!pip install -U goatools > /dev/null 2>&1
import os; os.kill(os.getpid(),9)


**Note**:  This installation may take more than 200 seconds. Please, after it is finished, run the following cells


In [ ]:
import pandas as pd
import numpy as np
import csv,os
import editdistance
from bio_embeddings.embed import SeqVecEmbedder
from goatools.obo_parser import GODag
from goatools.gosubdag.gosubdag import GoSubDag
from tqdm.notebook import tqdm

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


Downloading the GitHub repository and setting the data directory

In [ ]:
!git clone https://github.com/gamerino/DeeProtGO.git
os.chdir('DeeProtGO/data/')

Cloning into 'DeeProtGO'...
remote: Enumerating objects: 137, done.
remote: Total 137 (delta 0), reused 0 (delta 0), pack-reused 137
Receiving objects: 100% (137/137), 184.87 MiB | 12.56 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Checking out files: 100% (43/43), done.


### 1. Training data

#### 1.1 Loading protein data

Protein data for *NK* proteins from Eukarya organisms required for training DeeProtGO to predict BP terms is loaded from the 'trainingNKEukaBPInfo.tab' file. 

In [ ]:
eukaTrainingBPNK = pd.read_csv("raw/trainingNKEukaBPInfo.tab",
                                delimiter = "\t")


In [ ]:
eukaTrainingBPNK

,Entry,Taxon,Sequence,HasGOPT_1,HasGOPT0,PureNK,PosNKModel_P,Negative_P
0,A0A060X6Z0,8022,MPISSSSSSSTKSMRRAASELERSDSVTSPRFIGRRQSLIEDARKE...,False,False,False,False,True
1,A0A068FIK2,3635,MEVGGGSEECCVKVAVHVRPLIGDEKVQGCKDCVTVIPGKPQVQIG...,False,False,False,False,True
2,A0A075F932,8845,MVSESHHEALAAPPATTVAAAPPSNVTEPASPGGGGGKEDAFSKLK...,True,True,False,True,False
3,A0A096SRM5,4577,MAANGGDHTSARPHVVLLPSAGMGHLVPFARLAVALSEGHGCNVSV...,False,False,False,False,True
4,A0A0C5B5G6,9606,MRWQEMGYIFYPRKLR,False,True,False,True,False
...,...,...,...,...,...,...,...,...
18685,Q9Y4X0,9606,MAAGCCGVKKQKLSSSPPSGSGGGGGASSSSHCSGESQCRAGELGL...,False,False,False,False,True
18686,Q8IWA6,9606,MTKVPATKKLQSSPNSGAVRPFYASENLRQVPDKPMKSIKYMDKEI...,False,False,False,False,True
18687,Q8N5R6,9606,MAFRGPEPWVSASLLRQRLKAEEKTLDLEFEVLSVGFNEAGRYALR...,False,False,False,False,True
18688,Q8TAB7,9606,MERLCLQPGLLPTAVYLPHWERSSRDREEKEAPFFRLLSRRLMFCV...,False,False,False,False,True


This file contains eight columns describing the following aspects of training proteins:
 - *Entry*: The *Uniprot* protein Entry
 - *Taxon*: The *NCBI* unique identifier for the source organism
 - *Sequence*: The sequence of the canonical protein
 - *HasGOPT_1*: Logical indicating if the protein has annotations in GO-BP at the reference time ($t_{-1}$).
 - *HasGOPT0*: Logical indicating if the protein has annotations after the growth period ($t_{0}$).
 - *PureNK*: Logical indicating if the protein is a *NK* protein.
 - *PosNKModel_P*: Logical indicating if the protein is a positive case for the NK-BP-Euka model. Note proteins that are positives and not pureNK are those obtained during data augmentation. 
 - *Negative_P*: Logical indicating if the protein is not annotated at both $t_{-1}$ and $t_0$. 

In [ ]:
eukaTrainingBPNK.PosNKModel_P.value_counts()

False    13382
True      5308
Name: PosNKModel_P, dtype: int64

In [ ]:
(eukaTrainingBPNK.PureNK).value_counts()

False    18662
True        28
Name: PureNK, dtype: int64

Extracting proteins entry and sequence:

In [ ]:
NK_BPEntriesPos = eukaTrainingBPNK.loc[eukaTrainingBPNK.loc[:, 'PosNKModel_P'], 'Entry']
NK_BPEntriesNeg = eukaTrainingBPNK.loc[eukaTrainingBPNK.loc[:, 'Negative_P'], 'Entry']
NK_BPEntriesAll = NK_BPEntriesPos.append(NK_BPEntriesNeg).tolist()
NK_BPEntriesAll[:5]


['A0A075F932', 'A0A0C5B5G6', 'A0A0K3AV08', 'A0A0N9E2K8', 'A0A0R4IBK5']

In [ ]:
posSequences = eukaTrainingBPNK.loc[eukaTrainingBPNK.loc[:, 'PosNKModel_P'], 'Sequence'].tolist()
negSequences = eukaTrainingBPNK.loc[eukaTrainingBPNK.loc[:, 'Negative_P'], 'Sequence'].tolist()

In [ ]:
np.savetxt("processed/Training/PosEntries_Euka_BP.tab", NK_BPEntriesPos, 
           fmt = '%s', delimiter = "\t")
np.savetxt("processed/Training/NegEntries_Euka_BP.tab", NK_BPEntriesNeg, fmt='%s', delimiter="\t")


#### 1.2 Computing Levenshtein distance between protein sequences

The edit distance will be computed between all those proteins used for training DeeProtGO and those proteins that are positives for this particular task

In [ ]:
EditDistancePosProteins = np.zeros((len(posSequences), len(posSequences)))
EditDistanceNegProteins = np.zeros((len(negSequences), len(posSequences)))

In [ ]:
# Between positive proteins
for i in tqdm(range(len(posSequences))):
    for j in range(i+1,len(posSequences)):
        EditDistancePosProteins[i, j] = editdistance.eval(posSequences[i], posSequences[j])
        EditDistancePosProteins[j, i] = EditDistancePosProteins[i, j]


In [ ]:
EditDistancePosProteins[:4,:4]

array([[   0.,  406.,  851.,  474.],
       [ 406.,    0., 1043.,  583.],
       [ 851., 1043.,    0.,  828.],
       [ 474.,  583.,  828.,    0.]])

In [ ]:
# Between negative and positive proteins
for i in tqdm(range(len(negSequences))):
    for j in range(len(posSequences)):
        EditDistanceNegProteins[i , j] = editdistance.eval(negSequences[i], posSequences[j])

Combining the two edit distance matrices in a single matrix

In [ ]:
EditDistanceAll = np.concatenate((EditDistancePosProteins, EditDistanceNegProteins))

Trimming maximum length to the 99th percentile of proteins' length (2,353)

In [ ]:
EditDistanceAll[np.where(EditDistanceAll > 2353)] = 2353

Computing sequence similarity based on edit distance

In [ ]:
maxVal = np.max(EditDistanceAll)
simEditDistanceAll = 1 - EditDistanceAll/maxVal

Saving sequence similarity data

In [ ]:
pdLevSim = pd.DataFrame(simEditDistanceAll, index = NK_BPEntriesAll, columns = NK_BPEntriesPos)
pdLevSim.head()

Entry,A0A075F932,A0A0C5B5G6,A0A0K3AV08,A0A0N9E2K8,A0A0R4IBK5,A0AVF1,A0FGR8,A0FGR9,A0FLQ6,A0JMQ9,...,Q8GWB2,Q5SV66,Q8NBF2,Q8TF61,Q6PKX4,P0CU05,Q3TY65,Q96ME1,Q9FHK4,Q8IUR7
A0A075F932,1.000000,0.827454,0.638334,0.798555,0.0,0.813430,0.689333,0.704207,0.410965,0.759881,...,0.859754,0.860178,0.762006,0.703782,0.855504,0.844879,0.847004,0.725882,0.791755,0.778156
A0A0C5B5G6,0.827454,1.000000,0.556736,0.752231,0.0,0.771356,0.615385,0.630259,0.300892,0.702507,...,0.886528,0.871653,0.698258,0.634509,0.866128,0.829154,0.823204,0.664683,0.745431,0.720357
A0A0K3AV08,0.638334,0.556736,1.000000,0.648109,0.0,0.645134,0.639184,0.643009,0.461113,0.647684,...,0.624734,0.628134,0.655759,0.646409,0.630259,0.636634,0.643009,0.649809,0.651934,0.651509
A0A0N9E2K8,0.798555,0.752231,0.648109,1.000000,0.0,0.790055,0.696983,0.703782,0.435189,0.759456,...,0.798980,0.796855,0.752656,0.710582,0.801105,0.801105,0.798980,0.731832,0.782830,0.770081
A0A0R4IBK5,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
pdLevSim.to_hdf('processed/Training/LevSim_BP_Euka.h5', key = "df", mode = "w")

#### 1.3 Generating sequence embeddings

The [SeqVec](https://github.com/sacdallago/bio_embeddings) model is used for obtaining a dense representation of protein sequences.

In [ ]:
embedder = SeqVecEmbedder()

weights.hdf5: 374MB [08:23, 744kB/s]                                
options.json: 8.19kB [00:01, 7.00kB/s]


In [ ]:
protSeq = posSequences[0]
embedding = embedder.embed(protSeq)
embedder.reduce_per_protein(embedding)

array([ 0.00349385,  0.15905352, -0.16662982, ..., -0.17260969,
       -0.07606463, -0.001355  ], dtype=float32)

In [ ]:
# Embeddings for positive proteins
AllEmbPosProt = np.empty((len(posSequences), 1024))

for i in tqdm(range(len(posSequences))):
    protSeq = posSequences[i]
    embedding = embedder.embed(protSeq) # SeqVec at the AA level
    AllEmbPosProt[i, : ] = embedder.reduce_per_protein(embedding) # Reducing at the protein level


In [ ]:
# Embeddings for negative proteins
AllEmbNegProt = np.empty((len(negSequences), 1024))

for i in tqdm(range(len(negSequences))):
    protSeq = negSequences[i]
    embedding = embedder.embed(protSeq) # SeqVec at the AA level
    AllEmbNegProt[i, : ] = embedder.reduce_per_protein(embedding) # Reducing at the protein level


In [ ]:
pdEmb = pd.DataFrame(np.concatenate((AllEmbPosProt, AllEmbNegProt), axis = 0), index = NK_BPEntriesAll)
pdEmb

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
A0A075F932,0.003494,0.159053,-0.166630,0.075113,-0.156387,-0.104203,0.098892,-0.073795,-0.042953,-0.051893,...,0.017357,0.147743,0.261931,0.077637,-0.091048,-0.018328,0.016883,-0.172610,-0.076065,-0.001355
A0A0C5B5G6,-0.000224,0.052369,-0.069918,-0.136116,0.104354,0.089259,0.134537,-0.022843,-0.229596,-0.079265,...,0.124953,0.077058,-0.001223,-0.039764,-0.207887,-0.025197,-0.021677,0.003216,-0.011383,-0.009263
A0A0K3AV08,0.041304,0.016460,-0.048744,0.037136,0.025713,0.005405,0.116830,-0.008005,0.078220,-0.117836,...,0.029034,0.073025,-0.027677,0.199008,-0.022565,0.105686,-0.054289,-0.114955,-0.069601,-0.025704
A0A0N9E2K8,-0.023098,-0.074825,-0.065399,-0.062160,-0.028766,-0.024393,0.098963,0.063683,-0.004213,-0.046981,...,0.017419,0.048572,0.007422,-0.026016,0.075574,-0.086174,-0.052656,-0.103410,-0.042535,-0.073342
A0A0R4IBK5,-0.046586,-0.026175,-0.058654,-0.041342,-0.002978,-0.081237,0.121809,-0.049883,-0.023462,-0.084363,...,0.075727,0.083656,0.075023,0.017746,-0.108451,0.090608,0.020869,-0.029943,-0.030877,-0.069694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q9Y4X0,0.070359,-0.032819,-0.058298,0.034255,0.043293,0.026641,-0.017085,0.198669,0.117390,-0.098732,...,-0.066603,0.014317,-0.039960,0.064990,0.024282,0.061154,-0.023845,-0.010171,-0.012309,0.036144
Q8IWA6,-0.044569,-0.008961,-0.060896,-0.051849,0.035256,-0.030767,0.115316,-0.037560,-0.020422,-0.093968,...,0.009261,0.072149,0.045318,0.096674,-0.089745,0.066031,-0.000528,0.010366,0.066425,0.080046
Q8N5R6,0.021256,0.101291,-0.038480,-0.023916,0.010954,-0.062998,0.012282,0.101138,0.093903,-0.038706,...,-0.019549,0.054291,0.097541,0.081289,-0.034522,-0.001844,0.015700,-0.072382,0.032316,-0.052497
Q8TAB7,0.034143,-0.007347,-0.064990,-0.130398,-0.031139,-0.024653,0.111699,0.050865,0.029160,-0.089404,...,0.007578,0.074721,0.063619,0.046902,-0.076432,-0.016824,-0.043174,-0.046723,0.097002,0.077477


In [ ]:
pdEmb.to_hdf('processed/Training/Emb_BP_Euka.h5', key = "df", mode = "w")

#### 1.4 Formatting taxon data

One-hot enconded vectors are used for representing taxonomic information

In [ ]:
pdTaxon = pd.crosstab(eukaTrainingBPNK.loc[:, 'Entry'], eukaTrainingBPNK.loc[:, 'Taxon'], 
                             rownames = ['Proteins'], colnames = ['Taxon'])
pdTaxon = pdTaxon.reindex(NK_BPEntriesAll)
pdTaxon

Taxon,2762,2769,2903,3037,3039,3055,3311,3469,3490,3498,...,746128,756487,756488,763456,1077530,1108046,1176036,1221240,1234705,1260784
Proteins,,,,,,,,,,,,,,,,,,,,,
A0A075F932,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0A0C5B5G6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0A0K3AV08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0A0N9E2K8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0A0R4IBK5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q9Y4X0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Q8IWA6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Q8N5R6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
pdTaxon.to_hdf('processed/Training/Taxon_BP_Euka.h5', key = "df", mode = "w")

#### 1.5 Encoding GO terms

Propagated annotations at $t_0$ of positive proteins, referred to the time of reference $t_{-1}$ are provided.

In [ ]:
GOBPAnnotNKEukaT0 = pd.read_csv("intermediate/propAnnot_Train_Euka_BP.tab", 
                                delimiter="\t",header=None)
GOBPAnnotNKEukaT0

,0,1
0,A0A075F932,GO:0008150
1,A0A075F932,GO:0010646
2,A0A075F932,GO:0010817
3,A0A075F932,GO:0022414
4,A0A075F932,GO:0023051
...,...,...
112485,Q9LNP2,GO:0009987
112486,Q9LNP2,GO:0031930
112487,Q9LNP2,GO:0050789
112488,Q9LNP2,GO:0050794


In [ ]:
(GOBPAnnotNKEukaT0.loc[:,0]).value_counts()

P60896    218
P54130    198
P30645    162
P70390    150
P70289    144
         ... 
Q7ZVY5      3
P23060      3
P0CU04      3
Q09607      2
Q9ZUE0      2
Name: 0, Length: 5308, dtype: int64

Building a matrix with rows representing positive proteins and columns GO-BP terms

In [ ]:
fullOutProp = pd.crosstab(GOBPAnnotNKEukaT0.loc[:,0], GOBPAnnotNKEukaT0.loc[:,1], rownames=['Proteins'], colnames=['Terms'])
fullOutProp

Terms,GO:0000002,GO:0000003,GO:0000011,GO:0000018,GO:0000019,GO:0000023,GO:0000025,GO:0000027,GO:0000028,GO:0000038,...,GO:2001279,GO:2001280,GO:2001293,GO:2001295,GO:2001305,GO:2001307,GO:2001308,GO:2001310,GO:2001316,GO:2001317
Proteins,,,,,,,,,,,,,,,,,,,,,
A0A075F932,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0A0C5B5G6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0A0K3AV08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0A0N9E2K8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0A0R4IBK5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q9ZW30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Q9ZW96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
T1SFR8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Adding a zero-matrix for negative proteins.

In [ ]:
dfNegs = pd.DataFrame(np.zeros((len(NK_BPEntriesNeg), fullOutProp.shape[1])), index = NK_BPEntriesNeg,
                     columns = fullOutProp.columns)
netOut = pd.concat([fullOutProp, dfNegs], axis = 0).astype('int')
netOut = netOut.reindex(NK_BPEntriesAll)
netOut

Terms,GO:0000002,GO:0000003,GO:0000011,GO:0000018,GO:0000019,GO:0000023,GO:0000025,GO:0000027,GO:0000028,GO:0000038,...,GO:2001279,GO:2001280,GO:2001293,GO:2001295,GO:2001305,GO:2001307,GO:2001308,GO:2001310,GO:2001316,GO:2001317
A0A075F932,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0A0C5B5G6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0A0K3AV08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0A0N9E2K8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0A0R4IBK5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q9Y4X0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Q8IWA6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Q8N5R6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Q8TAB7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
netOut.to_hdf('processed/Training/netOut_BP_Euka.h5', key = "df", mode = "w")

#### 1.6 Building GO terms relationship matrix

After doing the prediction, DeeProtGO will perform the propagation of predicted scores in the same way as CAFA challenge. For doing this, it requires a file of relationships between terms that will be built here using [GOAtools](https://github.com/tanghaibao/goatools).

In [ ]:
oboFile = "extras/gene_ontology_edit.obo"

GOTerms = netOut.columns.tolist()

GOTerms[:5]

['GO:0000002', 'GO:0000003', 'GO:0000011', 'GO:0000018', 'GO:0000019']

In [ ]:
# Loading the GO
godag = GODag(oboFile, optional_attrs = {'relationship'})
# Selecting the GO terms from BP that will be predicted for the NK eukarya proteins
gosubdag_All = GoSubDag(GOTerms, godag, prt = None)
# Extracting relationships
goParents = gosubdag_All.rcntobj.go2parents
GORelations = np.zeros((len(GOTerms), len(GOTerms)), dtype = 'int')
# Building the relationship matrix

for i in tqdm(range(len(GOTerms))):
    GORelations[i,i] = 1
    term = GOTerms[i]
    if(term in goParents.keys()):
        termPar = list(goParents[term])
        if(len(termPar) >0 ):
            for p in termPar:
                j = np.where(np.array(GOTerms) == p)[0]
                if(len(j) > 0): 
                    GORelations[i,j] = 1


/home/gabriela/Insync/gmerino@sinc.unl.edu.ar/Google Drive/EBI-EMBL/GOAnnot/DeeProtGO/data/extras/gene_ontology_edit.obo: fmt(1.2) rel(2017-01-31) 46,174 GO Terms; optional_attrs(relationship)


In [ ]:
np.savetxt("processed/Training/GOTermsPropRel_Euka_BP_train.tab" , GORelations, 
           fmt = '%u', delimiter = "\t")


### 2. Benchmark data

#### 2.1 Loading protein data

Data for *NK* proteins from the CAFA3 benchmark dataset from eukarya organisms is available in the 'benchmarkNKEukaBPInfo.tab' file. 

In [ ]:
eukaBenchmarkBPNK = pd.read_csv("raw/benchmarkNKEukaBPInfo.tab",
                                delimiter = "\t")

In [ ]:
eukaBenchmarkBPNK

,CAFAID,Entry.name,Taxon,Sequence,HasGOPT0,HasGOPT1,PureNK,Negative_P
0,T100900000046,5HT3A_MOUSE,10090,MRLCIPQVLLALFLSMLTAPGEGSRRRATQEDTTQPALLRLSDHLL...,False,False,False,True
1,T100900000115,AB17B_MOUSE,10090,MNNLSFSELCCLFCCPPCPGKIASKLAFLPPDPTYTLMCDESGSRW...,False,True,True,False
2,T100900000116,AB17C_MOUSE,10090,MPEPGPRMNGFSLGELCWLFCCPPCPSRIAAKLAFLPPEPTYTVLA...,False,True,True,False
3,T100900000161,ABHD4_MOUSE,10090,MADDLEQQPQGWLSSWLPTWRPTSMSQLKNVEARILQCLQNKFLAR...,False,True,True,False
4,T100900000167,ABHDD_MOUSE,10090,MEKSWMLWSFIERWLLALASWSWALCRISLLPLIVTFHLYGGIVLL...,False,True,True,False
...,...,...,...,...,...,...,...,...
1019,T96060019458,ZBT7C_HUMAN,9606,MANDIDELIGIPFPNHSSEVLCSLNEQRHDGLLCDVLLVVQEQEYR...,False,True,True,False
1020,T96060019520,ZDH23_HUMAN,9606,MTQKGSMKPVKKKKTEEPELEPLCCCEYIDRNGEKNHVATCLCDCQ...,False,True,True,False
1021,T96060019755,ZN396_HUMAN,9606,MSAKLGKSSSLLTQTSEECNGILTEKMEEEEQTCDPDSSLHWSSSY...,False,True,True,False
1022,T96060019899,ZN609_HUMAN,9606,MSLSSGASGGKGVDANPVETYDSGDEWDIGVGNLIIDLDADLEKDQ...,False,True,True,False


This file contains eight columns describing the following aspects of training proteins:
 - *CAFAID*: The *CAFA* identifier for proteins in benchmark dataset.
 - *Enntry.name*: The UniProt Entry name of the protein.
 - *Taxon*: The *NCBI* unique identifier for the source organism.
 - *Sequence*: The sequence of the canonical protein.
 - *HasGOPT0*: Logical indicating if the protein has annotations in GO-BP at the reference time of the benchmark dataset ($t_{0}$).
 - *HasGOPT1*: Logical indicating if the protein has annotations after the growth period ($t_{1}$).
 - *PureNK*: Logical indicating if the protein is a *NK* protein.
 - *Negative_P*: Logical indicating if the protein is not annotated at both $t_{-1}$ and $t_0$. 

In [ ]:
eukaBenchmarkBPNK.PureNK.value_counts()


True     757
False    267
Name: PureNK, dtype: int64

Positive cases here are those *NK* benchmark proteins that gained annotations for BP during the growth period meanwhile negative cases are those *NK* benchmark proteins that did not gained annotations for this ontology during that annotation period

In [ ]:
NK_BP_BenchEntriesPos = eukaBenchmarkBPNK.loc[eukaBenchmarkBPNK.loc[:, 'PureNK'], 'Entry.name']
NK_BP_BenchEntriesNeg = eukaBenchmarkBPNK.loc[eukaBenchmarkBPNK.loc[:, 'Negative_P'], 'Entry.name']
NK_BP_BenchEntriesAll = NK_BP_BenchEntriesPos.append(NK_BP_BenchEntriesNeg).tolist()
NK_BP_BenchEntriesAll[:5]

['AB17B_MOUSE', 'AB17C_MOUSE', 'ABHD4_MOUSE', 'ABHDD_MOUSE', 'ADPGK_MOUSE']

In [ ]:
posSequencesBench = eukaBenchmarkBPNK.loc[eukaBenchmarkBPNK.loc[:, 'PureNK'], 'Sequence'].tolist()
negSequencesBench = eukaBenchmarkBPNK.loc[eukaBenchmarkBPNK.loc[:, 'Negative_P'], 'Sequence'].tolist()

In [ ]:
np.savetxt("processed/Benchmark/PosEntries_Euka_BP.tab", NK_BP_BenchEntriesPos, 
           fmt = '%s', delimiter = "\t")
np.savetxt("processed/Benchmark/NegEntries_Euka_BP.tab", NK_BP_BenchEntriesNeg, 
           fmt='%s', delimiter="\t")

#### 2.2 Computing Levensthein distance

The edit distance will be computed between NK-BP benchmark proteins and those positives proteins in the training dataset:

In [ ]:
EditDistPosProteinsBenchTrain = np.zeros((len(posSequencesBench), len(posSequences)))
EditDistNegProteinsBenchTrain = np.zeros((len(negSequencesBench), len(posSequences)))

In [ ]:
# Between positive proteins
for i in tqdm(range(len(posSequencesBench))):
    for j in range(len(posSequences)):
        EditDistPosProteinsBenchTrain[i , j] = editdistance.eval(posSequencesBench[i], posSequences[j])

In [ ]:
EditDistPosProteinsBenchTrain[:4, :4]

array([[332., 272., 888., 472.],
       [340., 304., 874., 470.],
       [340., 327., 870., 467.],
       [336., 321., 868., 467.]])

In [ ]:
# Between negative and positive proteins
for i in tqdm(range(len(negSequencesBench))):
    for j in range(len(posSequences)):
        EditDistNegProteinsBenchTrain[i , j] = editdistance.eval(negSequencesBench[i], posSequences[j])

Combining two matrices in a single matrix

In [ ]:
EditDistanceAllBench = np.concatenate((EditDistPosProteinsBenchTrain, EditDistNegProteinsBenchTrain))

Trimming maximum length to 2,353

In [ ]:
EditDistanceAllBench[np.where(EditDistanceAllBench >2353)]=2353

Computing sequence similarity based on edit distance

In [ ]:
maxVal = np.max(EditDistanceAllBench)
simEditDistanceAllBench = 1 - EditDistanceAllBench/maxVal

Saving sequence similarity data

In [ ]:
pdLevSim = pd.DataFrame(simEditDistanceAllBench, index = NK_BP_BenchEntriesAll, columns = NK_BPEntriesPos)
pdLevSim.head()

Entry,A0A075F932,A0A0C5B5G6,A0A0K3AV08,A0A0N9E2K8,A0A0R4IBK5,A0AVF1,A0FGR8,A0FGR9,A0FLQ6,A0JMQ9,...,Q8GWB2,Q5SV66,Q8NBF2,Q8TF61,Q6PKX4,P0CU05,Q3TY65,Q96ME1,Q9FHK4,Q8IUR7
AB17B_MOUSE,0.858904,0.884403,0.622609,0.799405,0.0,0.815980,0.678283,0.693583,0.386315,0.757331,...,0.896303,0.889503,0.751381,0.695708,0.886953,0.863153,0.856354,0.724182,0.794730,0.774331
AB17C_MOUSE,0.855504,0.870803,0.628559,0.800255,0.0,0.814705,0.683383,0.697408,0.390140,0.759881,...,0.891628,0.883978,0.754356,0.704632,0.883553,0.858904,0.857629,0.726732,0.795155,0.773056
ABHD4_MOUSE,0.855504,0.861028,0.630259,0.801530,0.0,0.816830,0.682958,0.699108,0.398215,0.762431,...,0.883978,0.879303,0.757756,0.698258,0.878028,0.856354,0.852104,0.727157,0.793030,0.776031
ABHDD_MOUSE,0.857204,0.863578,0.631109,0.801530,0.0,0.818105,0.685508,0.699108,0.398215,0.757756,...,0.884828,0.881853,0.759456,0.698258,0.879303,0.857204,0.855929,0.728432,0.792605,0.779006
ADPGK_MOUSE,0.831279,0.796005,0.647259,0.796430,0.0,0.810880,0.703782,0.711857,0.422439,0.765406,...,0.835954,0.835954,0.762856,0.713557,0.838079,0.828729,0.830429,0.739482,0.796855,0.778581


In [ ]:
pdLevSim.to_hdf('processed/Benchmark/LevSim_BP_Euka.h5', key = "df", mode = "w")

#### 2.3 Generating sequence embeddings

Same as for training proteins, the SeqVec model is used for obtaining a dense representation of protein sequences.

In [ ]:
if not 'embedder' in locals():
    embedder = SeqVecEmbedder() 
    

In [ ]:
# only for model callibration
protSeq = posSequencesBench[1]
embedding = embedder.embed(protSeq)
embedd_red = embedder.reduce_per_protein(embedding)
display(embedd_red)

array([-0.0336005 , -0.00748452, -0.09418942, ..., -0.01301473,
       -0.13464893, -0.08227421], dtype=float32)

In [ ]:
# Positive proteins
AllEmbPosProt = np.empty((len(posSequencesBench), 1024))

for i in tqdm(range(len(posSequencesBench))):
    protSeq = posSequencesBench[i]
    embedding = embedder.embed(protSeq) # SeqVec at the AA level
    AllEmbPosProt[i, : ] = embedder.reduce_per_protein(embedding) # Reducing at the protein level

In [ ]:
# Negative proteins
AllEmbNegProt = np.empty((len(negSequencesBench), 1024))

for i in tqdm(range(len(negSequencesBench))):
    protSeq = negSequences[i]
    embedding = embedder.embed(protSeq) # SeqVec at the AA level
    AllEmbNegProt[i, : ] = embedder.reduce_per_protein(embedding) # Reducing at the protein level


Saving embeddings data

In [ ]:
pdEmb = pd.DataFrame(np.concatenate((AllEmbPosProt, AllEmbNegProt), axis = 0), index = NK_BP_BenchEntriesAll)
pdEmb.to_hdf('processed/Benchmark/Emb_BP_Euka.h5', key = "df", mode = "w")


#### 2.4 Formatting taxon data
One-hot enconded vectors are used for representing taxonomic information

In [ ]:
NK_BP_taxon_train = pd.unique(eukaTrainingBPNK.loc[:, 'Taxon'].sort_values())

pdTaxonBench = pd.crosstab(eukaBenchmarkBPNK.loc[:, 'Entry.name'], eukaBenchmarkBPNK.loc[:, 'Taxon'], 
                             rownames = ['Proteins'], colnames = ['Taxon'])
pdTaxonBench = pdTaxonBench.reindex(NK_BP_BenchEntriesAll, axis = 0, fill_value = 0)


Taxon data must be referred to all the taxons that were observed in the training set

In [ ]:
pdTaxonBench = pdTaxonBench.reindex(NK_BP_taxon_train, axis = 1, fill_value = 0)
pdTaxonBench

Taxon,2762,2769,2903,3037,3039,3055,3311,3469,3490,3498,...,746128,756487,756488,763456,1077530,1108046,1176036,1221240,1234705,1260784
Proteins,,,,,,,,,,,,,,,,,,,,,
AB17B_MOUSE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AB17C_MOUSE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ABHD4_MOUSE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ABHDD_MOUSE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ADPGK_MOUSE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TOMT_HUMAN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TPC3L_HUMAN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
UBP29_HUMAN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
pdTaxonBench.to_hdf('processed/Benchmark/Taxon_BP_Euka.h5', key = "df", mode = "w")


#### 2.5 Encoding GO terms

Propagated annotations at $t_1$ of positive proteins, considering only those gained during the growth period ($t_0$, $t_1$) are provided.

In [ ]:
GOBPAnnotNKEukaBenchT1 = pd.read_csv("intermediate/propAnnot_Bench_Euka_BP.tab", 
                                delimiter = "\t", header = None)
GOBPAnnotNKEukaBenchT1


,0,1
0,3HIDH_ARATH,GO:0006082
1,3HIDH_ARATH,GO:0006520
2,3HIDH_ARATH,GO:0006551
3,3HIDH_ARATH,GO:0006573
4,3HIDH_ARATH,GO:0006807
...,...,...
19492,ZN668_MOUSE,GO:1903506
19493,ZN668_MOUSE,GO:1903507
19494,ZN668_MOUSE,GO:2000112
19495,ZN668_MOUSE,GO:2000113


In [ ]:
(GOBPAnnotNKEukaBenchT1.loc[:,0]).value_counts()

IRGM_HUMAN     162
CAR16_HUMAN    160
MFN1_RAT       155
TSLP_HUMAN     144
PDE12_HUMAN    106
              ... 
RANB9_DANRE      4
SNN_HUMAN        4
IMDH2_DANRE      3
ATG17_DICDI      3
G13B_DICDI       1
Name: 0, Length: 757, dtype: int64

One-hot enconded vectors are used for representing the propagated GO-BP terms of positive proteins.



In [ ]:
fullOutPropBench = pd.crosstab(GOBPAnnotNKEukaBenchT1.loc[:, 0], GOBPAnnotNKEukaBenchT1.loc[:, 1], 
                               rownames = ['Proteins'], colnames = ['Terms'])

The obtained table need to be expanded in columns in order to obtain a full representation of all BP terms learnt during model training

In [ ]:
# reindex using GOTerms of the fullOutProp DF

GOTerms_Euka_NK_BP = fullOutProp.columns

fullOutPropBench = fullOutPropBench.reindex(GOTerms_Euka_NK_BP.tolist(), axis = 1 , fill_value = 0)
fullOutPropBench = fullOutPropBench.astype('int')
fullOutPropBench

Terms,GO:0000002,GO:0000003,GO:0000011,GO:0000018,GO:0000019,GO:0000023,GO:0000025,GO:0000027,GO:0000028,GO:0000038,...,GO:2001279,GO:2001280,GO:2001293,GO:2001295,GO:2001305,GO:2001307,GO:2001308,GO:2001310,GO:2001316,GO:2001317
Proteins,,,,,,,,,,,,,,,,,,,,,
3HIDH_ARATH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AB17A_RAT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AB17B_MOUSE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AB17C_HUMAN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AB17C_MOUSE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZN608_MOUSE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZN609_HUMAN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZN609_MOUSE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Adding a zero-matrix for negative proteins.

In [ ]:
dfNegs = pd.DataFrame(np.zeros((len(NK_BP_BenchEntriesNeg), fullOutPropBench.shape[1])), 
                      index = NK_BP_BenchEntriesNeg, columns = fullOutPropBench.columns)
netOut = pd.concat([fullOutPropBench, dfNegs], axis = 0)
netOut = netOut.astype('int')
netOut = netOut.reindex(NK_BP_BenchEntriesAll)
netOut


Terms,GO:0000002,GO:0000003,GO:0000011,GO:0000018,GO:0000019,GO:0000023,GO:0000025,GO:0000027,GO:0000028,GO:0000038,...,GO:2001279,GO:2001280,GO:2001293,GO:2001295,GO:2001305,GO:2001307,GO:2001308,GO:2001310,GO:2001316,GO:2001317
AB17B_MOUSE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AB17C_MOUSE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ABHD4_MOUSE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ABHDD_MOUSE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ADPGK_MOUSE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TOMT_HUMAN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TPC3L_HUMAN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
UBP29_HUMAN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
VISTA_HUMAN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
netOut.to_hdf('processed/Benchmark/netOut_BP_Euka.h5', key = "df", mode = "w")